# 제 2 유형 연습하기 - 당뇨 진척 정도 (회귀)

## 데이터 분석 순서

### 1. 라이브러리 및 데이터 확인
### 2. 데이터 탐색(EDA)
### 3. 데이터 전처리 및 분리
### 4. 모델링 및 성능평가
### 5. 예측값 제출

### 1. 라이브러리 및 데이터 확인
##### 필수 라이브러리 - pandas / numpy

In [47]:
import pandas as pd
import numpy as np

from sklearn.datasets import load_diabetes

# diabetes 데이터셋 로드
diabetes = load_diabetes()

x = pd.DataFrame(diabetes.data, columns=diabetes.feature_names)
y = pd.DataFrame(diabetes.target)

# 실기시험 데이터 셋으로 세팅하기
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=2023)

x_test = pd.DataFrame(x_test.reset_index())
x_train = pd.DataFrame(x_train.reset_index())
y_train = pd.DataFrame(y_train.reset_index())

# index 컬럼의 이름을 index에서 cust_id로 바꾼거
x_test.rename(columns={'index': 'cust_id'}, inplace=True)
x_train.rename(columns={'index': 'cust_id'}, inplace=True)
# 지금 열 두개 있는데, 열 두개의 이름을 각각 cust_id, target으로 바꿈
y_train.columns = ['cust_id', 'target']

## 😇 당뇨병 환자의 질병 진행 정도를 예측해보자
### - 데이터의 결측치, 이상치, 변수들에 대해 전처리하고
### - 회귀 모델을 사용하여 Rsq, MSE 값을 산출하시오.
### - 제출은 cust_id, target 변수를 가진 dataframe 형태로 제출하시오.

In [34]:
# 데이터 설명
print(diabetes.DESCR)

.. _diabetes_dataset:

Diabetes dataset
----------------

Ten baseline variables, age, sex, body mass index, average blood
pressure, and six blood serum measurements were obtained for each of n =
442 diabetes patients, as well as the response of interest, a
quantitative measure of disease progression one year after baseline.

**Data Set Characteristics:**

  :Number of Instances: 442

  :Number of Attributes: First 10 columns are numeric predictive values

  :Target: Column 11 is a quantitative measure of disease progression one year after baseline

  :Attribute Information:
      - age     age in years
      - sex
      - bmi     body mass index
      - bp      average blood pressure
      - s1      tc, total serum cholesterol
      - s2      ldl, low-density lipoproteins
      - s3      hdl, high-density lipoproteins
      - s4      tch, total cholesterol / HDL
      - s5      ltg, possibly log of serum triglycerides level
      - s6      glu, blood sugar level

Note: Each of these 1

### 2. 데이터 탐색 (EDA)

In [35]:
# 데이터의 행/열 확인
print(x_train.shape)
print(x_test.shape)
print(y_train.shape)

(353, 11)
(89, 11)
(353, 2)


In [11]:
# 초기 데이터 확인
print(x_train.head())
print(x_test.head())
print(y_train.head())

   cust_id       age       sex       bmi        bp        s1        s2  \
0        4  0.005383 -0.044642 -0.036385  0.021872  0.003935  0.015596   
1      318  0.088931 -0.044642  0.006728  0.025315  0.030078  0.008707   
2      301 -0.001882  0.050680 -0.024529  0.052858  0.027326  0.030001   
3      189 -0.001882 -0.044642 -0.066563  0.001215 -0.002945  0.003070   
4      288  0.070769  0.050680 -0.016984  0.021872  0.043837  0.056305   

         s3        s4        s5        s6  
0  0.008142 -0.002592 -0.031988 -0.046641  
1  0.063367 -0.039493  0.009434  0.032059  
2  0.030232 -0.002592 -0.021395  0.036201  
3  0.011824 -0.002592 -0.020292 -0.025930  
4  0.037595 -0.002592 -0.070209 -0.017646  
   cust_id       age       sex       bmi        bp        s1        s2  \
0      280  0.009016  0.050680  0.018584  0.039087  0.017694  0.010586   
1      412  0.074401 -0.044642  0.085408  0.063187  0.014942  0.013091   
2       68  0.038076  0.050680 -0.029918 -0.040099 -0.033216 -0.02417

In [19]:
# 변수명과 데이터 타입이 매칭이 되는지, 결측치가 있는지 확인
# dtypes는 변수의 type만 나오고, info()는 정보 거의다 나옴 - null 값 있는지 없는지도 나옴
print(x_train.info())
print(x_test.info())
print(y_train.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 353 entries, 0 to 352
Data columns (total 11 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   cust_id  353 non-null    int64  
 1   age      353 non-null    float64
 2   sex      353 non-null    float64
 3   bmi      353 non-null    float64
 4   bp       353 non-null    float64
 5   s1       353 non-null    float64
 6   s2       353 non-null    float64
 7   s3       353 non-null    float64
 8   s4       353 non-null    float64
 9   s5       353 non-null    float64
 10  s6       353 non-null    float64
dtypes: float64(10), int64(1)
memory usage: 30.5 KB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 89 entries, 0 to 88
Data columns (total 11 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   cust_id  89 non-null     int64  
 1   age      89 non-null     float64
 2   sex      89 non-null     float64
 3   bmi      89 non-null     float64
 4   bp      

In [16]:
# y_train에 target 변수 이외의 변수가 있을 수 없지만, 분석을 위해서는 target 변수만 사용해야한다.

print(x_train.isnull().sum())
print(x_test.isnull().sum())
print(y_train.isnull().sum())

cust_id    0
age        0
sex        0
bmi        0
bp         0
s1         0
s2         0
s3         0
s4         0
s5         0
s6         0
dtype: int64
cust_id    0
age        0
sex        0
bmi        0
bp         0
s1         0
s2         0
s3         0
s4         0
s5         0
s6         0
dtype: int64
cust_id    0
target     0
dtype: int64


In [21]:
# x_train과 x_test 데이터의 기초 통계량을 잘 비교해보세요.
# 여기선 스케일링이 다 되어 있어서 확인할 게 없음!!
print(x_train.describe().T)
print(x_test.describe())
print(y_train.describe())

         count        mean         std       min         25%         50%  \
cust_id  353.0  212.634561  126.668903  0.000000  105.000000  210.000000   
age      353.0    0.000804    0.047617 -0.107226   -0.038207    0.005383   
sex      353.0    0.000724    0.047673 -0.044642   -0.044642   -0.044642   
bmi      353.0    0.000640    0.048141 -0.084886   -0.035307   -0.006206   
bp       353.0   -0.000326    0.046585 -0.112399   -0.033213   -0.005670   
s1       353.0    0.001179    0.047891 -0.126781   -0.033216   -0.002945   
s2       353.0    0.001110    0.048248 -0.115613   -0.029184   -0.001314   
s3       353.0   -0.000452    0.048600 -0.102307   -0.039719   -0.006584   
s4       353.0    0.000901    0.048045 -0.076395   -0.039493   -0.002592   
s5       353.0    0.001446    0.047160 -0.126097   -0.033246    0.000272   
s6       353.0    0.000589    0.048122 -0.137767   -0.034215    0.003064   

                75%         max  
cust_id  322.000000  441.000000  
age        0.038076

In [25]:
# y_train 데이터도 구체적으로 살펴보기
print(y_train.head())

   cust_id  target
0        4   135.0
1      318   109.0
2      301    65.0
3      189    79.0
4      288    80.0


In [26]:
# y 데이터가 연속형 변수다보니까 describe로 확인, 원래 분류형 모델에서는 value_counts() 사용  - target에 대한 기초 통계량 확인하면 됨( Min, max 값)
print(y_train.describe().T)

         count        mean         std   min    25%    50%    75%    max
cust_id  353.0  212.634561  126.668903   0.0  105.0  210.0  322.0  441.0
target   353.0  152.943343   75.324692  37.0   90.0  141.0  208.0  346.0


### 3. 데이터 전처리 및 분리
#### 1) 결측치 / 2) 이상치 / 3) 변수 처리하기

In [27]:
# 결측치 확인
print(x_train.isnull().sum())
print(x_test.isnull().sum())
print(y_train.isnull().sum())

cust_id    0
age        0
sex        0
bmi        0
bp         0
s1         0
s2         0
s3         0
s4         0
s5         0
s6         0
dtype: int64
cust_id    0
age        0
sex        0
bmi        0
bp         0
s1         0
s2         0
s3         0
s4         0
s5         0
s6         0
dtype: int64
cust_id    0
target     0
dtype: int64


In [ ]:
# 만약 결측치가 있을 경우
# 결측치 제거 df = df.dropna()
# 참고사항
# df = df.dropna().shape()  # 행 기준으로 삭제

In [ ]:
# 결측치 대체 (평균값, 중앙값, 최빈값)
# 연속형 변수 - 중앙값 / 평균값
## df['변수명'].median() / df['변수명'].mean()

# 범주형 변수 - 최빈값
## df['변수명'].mode()

## df['변수명'] = df['변수명'].fillna(대체할 값)

In [ ]:
# 이상치 대체
# (참고) df['변수명'] = np.where(df['변수명'] >= 5, 대체할 값, df['변수명'])

In [28]:
# 변수 처리

# 불필요한 변수 제거
## df = df.drop(columns = ['변수1', '변수2'])
## df = df.drop(['변수1', '변수2'], axis=1)

# 필요시 변수 추가 (파생변수 생성)
## df['파생변수명'] = df['A'] * df['B']  # 파생변수 생성 수식

# 원핫 인코딩 (가변수 처리)
## x_train = pd.get_dummies(x_train)
## x_test = pd.get_dummies(x_test)
## print(x_train.info())
## print(x_test.info())

In [48]:
# 변수 처리

# 불필요한 변수(columns 제거(
# cust_id는 불필요한 변수이므로 제거합니다.
# 단, test 셋의 cust_id가 나중에 제출이 필요하기 때문에 별도로 저장

## ** copy로 해줘야 함 !! 만약 제출 형식에 cust_id를 제출해야한다면 복사해놓기 !!
cust_id = x_test['cust_id'].copy()

x_train = x_train.drop(columns=['cust_id'])  # drop(columns=['변수1', '변수2']
x_test = x_test.drop(columns=['cust_id'])

#### 데이터 분리

In [49]:
# 데이터를 훈련 세트와 검증용 세트로 분할 (80% 훈련, 20% 검증용)
# 하이퍼 파라미터 튜닝? 뭔지 모르겠지만 하지마라.. - train data를 과적합 했다
# 검증용 데이터 나눌때 stratify 넣는거 아님 !! - 분류 모델에서만 사용 (Classififier), 연속형 데이터는 층화 샘플링을 할 필요가 없기 때문에 stratify 옵션 주지 마라/
from sklearn.model_selection import train_test_split
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train['target'], test_size=0.2, random_state=23)

In [50]:
print(x_train.shape)
print(x_val.shape)
print(y_train.shape)
print(y_val.shape)

(282, 10)
(71, 10)
(282,)
(71,)


### 4. 모델링 및 성능평가

In [51]:
# 랜덤포레스트 모델 사용 (참고 - 분류 모델은 RandomForestClassifier)
from sklearn.ensemble import RandomForestRegressor  # 회귀
model = RandomForestRegressor(random_state=2023)
model.fit(x_train, y_train)  # train data 학습

RandomForestRegressor(random_state=2023)

In [52]:
# 모델을 사용하여 테스트 데이터 예측
# 학습된 모델에 예측된 y값을 구하기 !!
# 이건 validation set에 대한 예측값
y_pred = model.predict(x_val)

In [53]:
# 모델 성능 평가 (평균 제곱 오차 및 R-squared)
from sklearn.metrics import mean_squared_error, r2_score
mse = mean_squared_error(y_val, y_pred)  # (실제값, 예측값)
r2 = r2_score(y_val, y_pred)

In [54]:
# MSE(mean_squared_error)
print(mse)

2563.5036816901406


In [55]:
# R2 score (R-squared) - 성능이 50% 정도.... 원래 당뇨병 데이터는 성능이 좋지 않음 !!
# 실제 시험에서는 원래 데이터에 더 좋은 모델을 쓰지 않아도 이정도가 성능 맥스일수도 있다 - 그냥 랜덤포레스트 쓰기
print(r2)

0.5250663004710372


In [56]:
# RMSE
rmse = mse ** 0.5  # ** mse에 루트 씌운거 ** 은 제곱승임 !! (mse에 1/2승)
print(rmse)

50.631054518843875


### 5. 예측값 제출
#### 주의 !!! x_test를 모델에 넣어 나온 예측값을 제출해야함

In [57]:
# 실기시험 안내사항
# 아래 코드 예측 변수와 수험번호를 개인별로 변경하여 활용
# pd.DataFrame({'cust_id':cust_id, 'target': y_result}).to_csv('0030000.csv', index=False)

# 모델을 사용하여 테스트 데이터 예측
y_result = model.predict(x_test)
result = pd.DataFrame({'cust_id': cust_id, 'target': y_result})
print(result[:5])

   cust_id  target
0      280  186.51
1      412  255.92
2       68   77.97
3      324  185.64
4      101  111.14


In [ ]:
# tip : 데이터를 저장한 다음, 불러와서 제대로 제출했는지 확인해보자.
# pd.DataFrame({'result':y_result}).to_csv('수험번호.csv', index=False)
# d2 = pd.read_csv('수험번호.csv')
# pring(df2.head())

## 제 2유형 연습하기 - 팁 예측하기 (회귀)

### 데이터 분석 순서
#### 1. 라이브러리 및 데이터 확인
#### 2. 데이터 탐색(EDA)
#### 3. 데이터 전처리 및 분리
#### 4. 모델링 및 성능 평가
#### 5. 예측값 제출

### 1. 라이브러리 및 데이터 확인

In [58]:
import pandas as pd
import numpy as np

In [60]:
# 실기 시험 데이터셋으로 세팅하기

import seaborn as sns
df = sns.load_dataset('tips')

x = df.drop(['tip'], axis=1)
y = df['tip']

from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=2023)

x_train = pd.DataFrame(x_train.reset_index())
x_test = pd.DataFrame(x_test.reset_index())
y_train = pd.DataFrame(y_train.reset_index())

In [61]:
x_train

,index,total_bill,sex,smoker,day,time,size
0,158,13.39,Female,No,Sun,Dinner,2
1,186,20.90,Female,Yes,Sun,Dinner,3
2,21,20.29,Female,No,Sat,Dinner,2
3,74,14.73,Female,No,Sat,Dinner,2
4,43,9.68,Male,No,Sun,Dinner,2
...,...,...,...,...,...,...,...
190,161,12.66,Male,No,Sun,Dinner,2
191,183,23.17,Male,Yes,Sun,Dinner,4
192,198,13.00,Female,Yes,Thur,Lunch,2
193,25,17.81,Male,No,Sat,Dinner,4
